In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
生成图片浏览 HTML（缩略图网格 + 灯箱查看），并支持：
- 大图滚轮缩放（以鼠标位置为锚点）
- 拖拽平移
- 按钮：放大/缩小/适配/1:1
- 双击：适配 <-> 1:1 切换
- 键盘：左右切换、Esc 关闭、+/- 缩放、F/0 适配
- 点击黑色背景关闭（包含：屏幕 backdrop + stage 内空白黑底）

用法：
  python gallery.py /path/to/images -o /path/to/out/gallery.html -t "My Gallery"
默认输出：
  不指定 -o 时输出到图片目录下 gallery.html
"""

from pathlib import Path
import html
import mimetypes
import argparse

IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".gif", ".bmp", ".svg"}

HTML_TEMPLATE = """<!doctype html>
<html lang="zh-CN">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>{title}</title>
  <style>
    :root {{
      --gap: 12px;
      --thumb-h: 180px;
      --bg: #0b0c10;
      --card: #111218;
      --text: #e8e8ea;
      --muted: #a9abb3;
      --border: rgba(255,255,255,0.10);
    }}
    body {{
      margin: 0;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, "Helvetica Neue", Arial;
      background: var(--bg);
      color: var(--text);
    }}
    header {{
      position: sticky;
      top: 0;
      backdrop-filter: blur(8px);
      background: rgba(11,12,16,0.75);
      border-bottom: 1px solid var(--border);
      padding: 12px 16px;
      z-index: 10;
    }}
    .title {{
      font-size: 16px;
      font-weight: 600;
      margin: 0 0 6px 0;
    }}
    .meta {{
      font-size: 12px;
      color: var(--muted);
      display: flex;
      gap: 14px;
      flex-wrap: wrap;
    }}
    .wrap {{
      padding: 16px;
      max-width: 1400px;
      margin: 0 auto;
    }}
    .grid {{
      display: grid;
      grid-template-columns: repeat(auto-fill, minmax(220px, 1fr));
      gap: var(--gap);
    }}

    /* Card as button */
    .cardbtn {{
      appearance: none;
      border: none;
      background: transparent;
      padding: 0;
      text-align: left;
      cursor: zoom-in;
      color: inherit;
    }}

    .card {{
      background: var(--card);
      border: 1px solid var(--border);
      border-radius: 14px;
      overflow: hidden;
      box-shadow: 0 6px 20px rgba(0,0,0,0.25);
      transition: transform .12s ease;
    }}
    .cardbtn:hover .card {{ transform: translateY(-2px); }}

    .thumb {{
      height: var(--thumb-h);
      display: flex;
      align-items: center;
      justify-content: center;
      background: #0f1016;
    }}
    .thumb img {{
      max-width: 100%;
      max-height: 100%;
      object-fit: contain;
      display: block;
      user-select: none;
      -webkit-user-drag: none;
    }}
    .info {{
      padding: 10px 12px 12px 12px;
      border-top: 1px solid var(--border);
    }}
    .name {{
      font-size: 12px;
      line-height: 1.35;
      word-break: break-all;
    }}
    .tag {{
      margin-top: 6px;
      font-size: 11px;
      color: var(--muted);
    }}

    .footer {{
      padding: 18px 0 10px 0;
      text-align: center;
      color: var(--muted);
      font-size: 12px;
    }}

    /* Lightbox */
    .lightbox {{
      position: fixed;
      inset: 0;
      display: none;
      z-index: 9999;
    }}
    .lightbox.open {{ display: block; }}
    .lb-backdrop {{
      position: absolute;
      inset: 0;
      background: rgba(0,0,0,0.75);
    }}
    .lb-panel {{
      position: absolute;
      inset: 24px;
      display: grid;
      grid-template-rows: auto 1fr auto;
      gap: 10px;
    }}
    .lb-top {{
      display: flex;
      justify-content: space-between;
      align-items: center;
      gap: 12px;
      color: var(--text);
      font-size: 13px;
    }}
    .lb-title {{
      overflow: hidden;
      text-overflow: ellipsis;
      white-space: nowrap;
      color: var(--muted);
    }}
    .lb-actions {{
      display: flex;
      gap: 8px;
      flex-shrink: 0;
    }}
    .lb-btn {{
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.8);
      color: var(--text);
      border-radius: 10px;
      padding: 8px 10px;
      cursor: pointer;
      font-size: 12px;
    }}
    .lb-btn:hover {{ filter: brightness(1.1); }}

    .lb-stage {{
      position: relative;
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.5);
      border-radius: 16px;
      overflow: hidden;
      display: flex;
      align-items: center;
      justify-content: center;
      touch-action: none; /* 允许我们接管滚轮/拖拽 */
    }}

    /* 大图：用 CSS 变量控制缩放/平移 */
    .lb-img{{
      --tx: 0px;
      --ty: 0px;
      --s: 1;

      max-width: none;
      max-height: none;
      object-fit: contain;

      position: absolute;
      left: 50%;
      top: 50%;
      transform: translate(calc(-50% + var(--tx)), calc(-50% + var(--ty))) scale(var(--s));
      transform-origin: center center;

      user-select: none;
      -webkit-user-drag: none;
      cursor: grab;
    }}
    .lb-stage.grabbing .lb-img {{ cursor: grabbing; }}

    .lb-bottom {{
      display: flex;
      justify-content: center;
      gap: 10px;
      color: var(--muted);
      font-size: 12px;
    }}
    .lb-hint {{
      opacity: 0.9;
    }}
    body.lb-lock {{
      overflow: hidden;
    }}
  </style>
</head>
<body>
<header>
  <div class="title">{title}</div>
  <div class="meta">
    <div>目录：{dir_name}</div>
    <div>图片数：{count}</div>
  </div>
</header>

<div class="wrap">
  <div class="grid">
    {cards}
  </div>
  <div class="footer">Generated by Python</div>
</div>

<!-- Lightbox -->
<div id="lightbox" class="lightbox" aria-hidden="true">
  <div class="lb-backdrop" id="lb-backdrop"></div>
  <div class="lb-panel" role="dialog" aria-modal="true" aria-label="Image viewer">
    <div class="lb-top">
      <div class="lb-title" id="lb-title"></div>
      <div class="lb-actions">
        <button class="lb-btn" id="lb-zoomout" type="button">−</button>
        <button class="lb-btn" id="lb-zoomin" type="button">+</button>
        <button class="lb-btn" id="lb-fit" type="button">适配</button>
        <button class="lb-btn" id="lb-actual" type="button">1:1</button>

        <button class="lb-btn" id="lb-prev" type="button">← 上一张</button>
        <button class="lb-btn" id="lb-next" type="button">下一张 →</button>
        <button class="lb-btn" id="lb-close" type="button">关闭 (Esc)</button>
      </div>
    </div>

    <div class="lb-stage">
      <img id="lb-img" class="lb-img" alt="">
    </div>

    <div class="lb-bottom">
      <div class="lb-hint">提示：滚轮缩放，拖拽平移，双击适配/1:1，左右键切换，Esc 关闭，点击黑色背景关闭</div>
    </div>
  </div>
</div>

<script>
(function() {{
  const items = Array.from(document.querySelectorAll('[data-lb="item"]'));
  const lightbox = document.getElementById('lightbox');
  const imgEl = document.getElementById('lb-img');
  const titleEl = document.getElementById('lb-title');

  const btnClose = document.getElementById('lb-close');
  const btnPrev = document.getElementById('lb-prev');
  const btnNext = document.getElementById('lb-next');
  const backdrop = document.getElementById('lb-backdrop');

  const btnZoomIn = document.getElementById('lb-zoomin');
  const btnZoomOut = document.getElementById('lb-zoomout');
  const btnFit = document.getElementById('lb-fit');
  const btnActual = document.getElementById('lb-actual');

  const stageEl = document.querySelector('.lb-stage');

  let idx = -1;

  // ===== Zoom/Pan state =====
  let scale = 1;
  let tx = 0;
  let ty = 0;
  let fitScale = 1;
  let userInteracted = false;

  const MIN_SCALE = 0.05;
  const MAX_SCALE = 16;

  function applyTransform() {{
    imgEl.style.setProperty('--s', String(scale));
    imgEl.style.setProperty('--tx', `${{tx}}px`);
    imgEl.style.setProperty('--ty', `${{ty}}px`);
  }}

  function clamp(v, a, b) {{ return Math.max(a, Math.min(b, v)); }}

  function fitToStage() {{
    const iw = imgEl.naturalWidth || 1;
    const ih = imgEl.naturalHeight || 1;

    const sw = stageEl.clientWidth || 1;
    const sh = stageEl.clientHeight || 1;

    fitScale = Math.min(sw / iw, sh / ih);
    fitScale = clamp(fitScale, MIN_SCALE, MAX_SCALE);

    scale = fitScale;
    tx = 0;
    ty = 0;
    userInteracted = false;
    applyTransform();
  }}

  function setActual() {{
    scale = 1;
    tx = 0;
    ty = 0;
    userInteracted = true;
    applyTransform();
  }}

  function zoomTo(newScale, clientX, clientY) {{
    newScale = clamp(newScale, MIN_SCALE, MAX_SCALE);

    const rect = stageEl.getBoundingClientRect();
    const cx = rect.left + rect.width / 2;
    const cy = rect.top + rect.height / 2;

    // 当前鼠标点在“图像坐标系(以图片中心为原点)”下的位置
    const dx = (clientX - cx - tx) / scale;
    const dy = (clientY - cy - ty) / scale;

    // 更新 scale 后，反推 tx/ty，让鼠标点尽量保持不动
    scale = newScale;
    tx = (clientX - cx) - dx * scale;
    ty = (clientY - cy) - dy * scale;

    userInteracted = true;
    applyTransform();
  }}

  function zoomBy(factor, anchorX, anchorY) {{
    zoomTo(scale * factor, anchorX, anchorY);
  }}

  function zoomAtCenter(factor) {{
    const rect = stageEl.getBoundingClientRect();
    zoomBy(factor, rect.left + rect.width / 2, rect.top + rect.height / 2);
  }}

  // ===== Lightbox navigation =====
  function setIndex(newIdx) {{
    if (!items.length) return;
    idx = (newIdx + items.length) % items.length;

    const el = items[idx];
    const src = el.getAttribute('data-src');
    const name = el.getAttribute('data-name') || '';

    imgEl.removeAttribute('src');
    imgEl.onload = () => {{
      fitToStage(); // 每张图打开默认适配
    }};

    imgEl.src = src;
    imgEl.alt = name;
    titleEl.textContent = `${{idx + 1}} / ${{items.length}}  ·  ${{name}}`;
  }}

  function openAt(i) {{
    setIndex(i);
    lightbox.classList.add('open');
    lightbox.setAttribute('aria-hidden', 'false');
    document.body.classList.add('lb-lock');

    stageEl.classList.remove('grabbing');
  }}

  function close() {{
    lightbox.classList.remove('open');
    lightbox.setAttribute('aria-hidden', 'true');
    document.body.classList.remove('lb-lock');

    imgEl.removeAttribute('src');

    scale = 1; tx = 0; ty = 0; fitScale = 1; userInteracted = false;
    applyTransform();
    stageEl.classList.remove('grabbing');
  }}

  function prev() {{ setIndex(idx - 1); }}
  function next() {{ setIndex(idx + 1); }}

  // ===== Bind card click =====
  items.forEach((el, i) => {{
    el.addEventListener('click', () => openAt(i));
  }});

  // ===== Buttons =====
  btnClose.addEventListener('click', close);
  backdrop.addEventListener('click', close);

  btnPrev.addEventListener('click', prev);
  btnNext.addEventListener('click', next);

  btnFit.addEventListener('click', fitToStage);
  btnActual.addEventListener('click', setActual);
  btnZoomIn.addEventListener('click', () => zoomAtCenter(1.25));
  btnZoomOut.addEventListener('click', () => zoomAtCenter(0.8));

  // ===== Wheel zoom =====
  stageEl.addEventListener('wheel', (e) => {{
    if (!lightbox.classList.contains('open')) return;
    e.preventDefault();

    const factor = (e.deltaY > 0) ? 0.9 : 1.1;
    zoomBy(factor, e.clientX, e.clientY);
  }}, {{ passive: false }});

  // ===== Drag to pan (Pointer Events) + 点击黑色空白关闭 =====
  let dragging = false;
  let startX = 0, startY = 0;
  let startTx = 0, startTy = 0;

  // 新增：拖拽判定 + 防误触关闭
  let moved = false;
  let lastDragAt = 0;

  stageEl.addEventListener('pointerdown', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    dragging = true;
    moved = false;
    startX = e.clientX;
    startY = e.clientY;
    startTx = tx;
    startTy = ty;

    stageEl.classList.add('grabbing');
    stageEl.setPointerCapture(e.pointerId);
  }});

  stageEl.addEventListener('pointermove', (e) => {{
    if (!dragging) return;

    const dx = e.clientX - startX;
    const dy = e.clientY - startY;

    // 如果移动超过阈值，认为是拖拽
    if (Math.abs(dx) + Math.abs(dy) > 2) moved = true;

    tx = startTx + dx;
    ty = startTy + dy;

    userInteracted = true;
    applyTransform();
  }});

  stageEl.addEventListener('pointerup', (e) => {{
    if (!dragging) return;
    dragging = false;
    stageEl.classList.remove('grabbing');
    try {{ stageEl.releasePointerCapture(e.pointerId); }} catch (_) {{}}

    // 记录拖拽结束时间，避免拖完立刻 click 触发关闭
    if (moved) lastDragAt = Date.now();
  }});

  stageEl.addEventListener('pointercancel', (e) => {{
    dragging = false;
    stageEl.classList.remove('grabbing');
    try {{ stageEl.releasePointerCapture(e.pointerId); }} catch (_) {{}}
  }});

  // 点击 stage 的空白区域关闭（点到图片不关）
  stageEl.addEventListener('click', (e) => {{
    if (!lightbox.classList.contains('open')) return;
    if (Date.now() - lastDragAt < 250) return; // 刚拖完不关
    if (e.target === stageEl) close();
  }});

  // ===== Double click: toggle fit <-> 1:1 =====
  stageEl.addEventListener('dblclick', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    if (Math.abs(scale - fitScale) < 1e-3) {{
      zoomTo(1, e.clientX, e.clientY); // 以双击点为锚点放到 1:1
    }} else {{
      fitToStage();
    }}
  }});

  // ===== Keyboard =====
  window.addEventListener('keydown', (e) => {{
    if (!lightbox.classList.contains('open')) return;

    if (e.key === 'Escape') close();
    else if (e.key === 'ArrowLeft') prev();
    else if (e.key === 'ArrowRight') next();
    else if (e.key === '+' || e.key === '=') zoomAtCenter(1.25);
    else if (e.key === '-' || e.key === '_') zoomAtCenter(0.8);
    else if (e.key.toLowerCase() === 'f') fitToStage();
    else if (e.key === '0') fitToStage();
  }});

  // ===== Resize: 如果用户还没手动缩放/拖拽，就保持适配 =====
  window.addEventListener('resize', () => {{
    if (!lightbox.classList.contains('open')) return;
    if (!userInteracted && imgEl.getAttribute('src')) {{
      fitToStage();
    }}
  }});
}})();
</script>

</body>
</html>
"""

CARD_TEMPLATE = """
<button type="button" class="cardbtn" data-lb="item" data-src="{src}" data-name="{name}">
  <div class="card">
    <div class="thumb">
      <img src="{src}" loading="lazy" alt="{alt}">
    </div>
    <div class="info">
      <div class="name">{name}</div>
      <div class="tag">{size}</div>
    </div>
  </div>
</button>
"""


def human_size(num_bytes: int) -> str:
    units = ["B", "KB", "MB", "GB", "TB"]
    size = float(num_bytes)
    i = 0
    while size >= 1024 and i < len(units) - 1:
        size /= 1024.0
        i += 1
    if i == 0:
        return f"{int(size)} {units[i]}"
    return f"{size:.2f} {units[i]}"


def is_image(p: Path) -> bool:
    if p.suffix.lower() in IMG_EXTS:
        return True
    mt, _ = mimetypes.guess_type(str(p))
    return (mt or "").startswith("image/")


def generate_gallery_html(img_dir: Path, out_html: Path, title: str | None = None) -> None:
    img_dir = img_dir.resolve()
    out_html = out_html.resolve()
    title = title or f"Image Gallery - {img_dir.name}"

    images = [p for p in img_dir.iterdir() if p.is_file() and is_image(p)]
    images.sort(key=lambda p: p.name.lower())

    cards = []
    for p in images:
        rel = p.relative_to(out_html.parent)
        rel_str = str(rel).replace("\\", "/")
        stat = p.stat()
        cards.append(
            CARD_TEMPLATE.format(
                src=html.escape(rel_str),
                alt=html.escape(p.name),
                name=html.escape(p.name),
                size=human_size(stat.st_size),
            )
        )

    out_html.parent.mkdir(parents=True, exist_ok=True)
    page = HTML_TEMPLATE.format(
        title=html.escape(title),
        dir_name=html.escape(str(img_dir)),
        count=len(images),
        cards="\n".join(cards),
    )
    out_html.write_text(page, encoding="utf-8")

In [2]:
img_dir = Path("E:\\project\\cloud_seg_compare_page\\20260128_compare")
out_html = Path("20260128.html")
title = '20260128 Cloud Segmentation Results'

if not img_dir.exists() or not img_dir.is_dir():
    raise SystemExit(f"Not a directory: {img_dir}")

generate_gallery_html(img_dir, out_html, title=title)
print(f"✅ Generated: {out_html.resolve()}")


✅ Generated: E:\project\cloud_seg_compare_page\20260128.html


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import argparse
import json
import os
from pathlib import Path

import numpy as np

try:
    import tifffile as tiff
except Exception:
    tiff = None

from PIL import Image


HTML_TEMPLATE = r"""
<!doctype html>
<html lang="zh-CN">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>卷帘对比</title>
  <style>
    :root { --bg:#0b0f14; --fg:#e6edf3; --muted:#9aa4af; --card:#111824; --line:#243042; }
    * { box-sizing: border-box; }
    body{
      margin:0; padding:18px;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, "PingFang SC","Microsoft YaHei";
      background: var(--bg); color: var(--fg);
    }
    .topbar{
      display:flex; flex-wrap:wrap; gap:10px; align-items:center;
      margin-bottom: 12px;
    }
    .card{
      background: var(--card);
      border: 1px solid var(--line);
      border-radius: 14px;
      padding: 12px;
    }
    label { color: var(--muted); font-size: 13px; }
    select, button, input{
      background: #0f1622; color: var(--fg);
      border: 1px solid var(--line);
      border-radius: 10px;
      padding: 8px 10px;
    }
    select, button { cursor: pointer; }
    button:hover, select:hover, input:hover { border-color: #3a4a64; }
    .spacer{ flex:1; }
    .hint{ color: var(--muted); font-size: 13px; line-height: 1.4; }
    .range{ width: 220px; }

    /* layout */
    .viewer-wrap{
      margin-top: 10px;
      display: grid;
      grid-template-columns: 500px 1fr;
      gap: 12px;
      align-items: start;
    }
    @media (max-width: 980px){
      .viewer-wrap{ grid-template-columns: 1fr; }
      .sidebar{ height: 260px !important; }
    }

    /* sidebar thumbs */
    .sidebar{
      height: min(78vh, 900px);
      overflow: hidden;
      padding: 10px;
    }
    .side-title{
      font-size: 13px;
      color: var(--muted);
      margin-bottom: 8px;
      display:flex;
      align-items:center;
      justify-content:space-between;
      gap:10px;
    }
    .side-title small{ color: var(--muted); opacity: .9; }
    .thumb-search{
      width: 100%;
      margin-bottom: 10px;
      outline: none;
    }
    .thumb-list{
      height: calc(100% - 26px - 46px);
      overflow: auto;
      display: grid;
      grid-template-columns: 1fr;
      gap: 8px;
      padding-right: 6px;
    }
    @media (max-width: 980px){
      .thumb-list{ height: calc(100% - 26px - 46px); }
    }

    .thumb-item{
      display: grid;
      grid-template-columns: 72px 1fr;
      gap: 10px;
      align-items: center;
      padding: 8px;
      border-radius: 12px;
      border: 1px solid var(--line);
      background: #0f1622;
      cursor: pointer;
    }
    .thumb-item:hover{ border-color:#3a4a64; }
    .thumb-item.active{
      border-color: rgba(255,255,255,0.65);
      box-shadow: 0 0 0 1px rgba(255,255,255,0.15) inset;
    }
    .thumb-item img{
      width: 72px;
      height: 54px;
      object-fit: contain;
      border-radius: 10px;
      background: #05070a;
      display:block;
    }
    .thumb-name{
      font-size: 13px;
      color: var(--fg);
      overflow: hidden;
      text-overflow: ellipsis;
      white-space: nowrap;
    }
    .thumb-sub{
      font-size: 12px;
      color: var(--muted);
      overflow:hidden;
      text-overflow: ellipsis;
      white-space: nowrap;
      margin-top: 2px;
    }

    /* viewer */
    .viewer{
      position: relative;
      width: 100%;
      height: min(78vh, 900px);
      border-radius: 16px;
      overflow: hidden;
      border: 1px solid var(--line);
      background: #05070a;
      user-select: none;
      touch-action: none;
    }
    .layer{
      position:absolute; inset:0;
      display:flex; align-items:center; justify-content:center;
    }
    .layer img{
      max-width: 100%;
      max-height: 100%;
      width: auto;
      height: auto;
      image-rendering: auto;
      pointer-events: none;
    }
    .top{
      clip-path: inset(0 calc(100% - var(--reveal)) 0 0);
    }
    .divider{
      position:absolute; top:0; bottom:0;
      left: var(--reveal);
      width: 2px;
      background: rgba(255,255,255,0.75);
      transform: translateX(-1px);
      pointer-events:none;
    }
    .handle{
      position:absolute;
      left: var(--reveal);
      top: 50%;
      transform: translate(-50%, -50%);
      width: 36px; height: 36px;
      border-radius: 18px;
      background: rgba(255,255,255,0.9);
      color:#0b0f14;
      display:flex; align-items:center; justify-content:center;
      font-weight: 700;
      pointer-events:none;
      box-shadow: 0 8px 30px rgba(0,0,0,0.35);
    }
    .meta{
      display:flex; flex-wrap:wrap; gap:10px; align-items:center;
      justify-content: space-between;
      margin-top: 10px;
      color: var(--muted);
      font-size: 13px;
    }
    .kbd{ padding:2px 6px; border:1px solid var(--line); border-radius:6px; background:#0f1622; color:var(--fg); }
    .badge{ padding:4px 8px; border:1px solid var(--line); border-radius:999px; }
  </style>
</head>

<body>
  <div class="topbar card">
    <div>
      <label>选择文件</label><br/>
      <select id="fileSelect"></select>
    </div>

    <div>
      <label>卷帘位置</label><br/>
      <input id="slider" class="range" type="range" min="0" max="100" value="50"/>
    </div>

    <div>
      <label>操作</label><br/>
      <button id="prevBtn">上一张</button>
      <button id="nextBtn">下一张</button>
      <button id="swapBtn">左右互换</button>
    </div>

    <div class="spacer"></div>
    <div class="hint">
      拖动分割线或滑块。快捷键：<span class="kbd">←</span>/<span class="kbd">→</span> 调卷帘，
      <span class="kbd">↑</span>/<span class="kbd">↓</span> 切图，<span class="kbd">S</span> 互换
    </div>
  </div>

  <div class="viewer-wrap">
    <!-- Sidebar -->
    <div class="sidebar card">
      <div class="side-title">
        <span>图片列表</span>
        <small id="sideCount"></small>
      </div>
      <input id="thumbSearch" class="thumb-search" placeholder="搜索文件名（支持模糊）..." />
      <div class="thumb-list" id="thumbList"></div>
    </div>

    <!-- Main -->
    <div>
      <div class="viewer" id="viewer" style="--reveal:50%;">
        <div class="layer bottom"><img id="imgB" alt="B"/></div>
        <div class="layer top"><img id="imgA" alt="A"/></div>
        <div class="divider"></div>
        <div class="handle">⇆</div>
      </div>

      <div class="meta card">
        <div>
          <span class="badge" id="leftLabel">左：Before</span>
          <span class="badge" id="rightLabel">右：Dehazed</span>
          <span style="margin-left:10px;" id="nameLabel"></span>
        </div>
        <div id="countLabel"></div>
      </div>
    </div>
  </div>

<script>
let manifest = null;
let idx = 0;
let swapped = false;

const sel = document.getElementById("fileSelect");
const slider = document.getElementById("slider");
const viewer = document.getElementById("viewer");
const imgA = document.getElementById("imgA");
const imgB = document.getElementById("imgB");
const nameLabel = document.getElementById("nameLabel");
const countLabel = document.getElementById("countLabel");
const leftLabel = document.getElementById("leftLabel");
const rightLabel = document.getElementById("rightLabel");

const thumbList = document.getElementById("thumbList");
const thumbSearch = document.getElementById("thumbSearch");
const sideCount = document.getElementById("sideCount");

let filteredIndices = null; // null = no filter, otherwise array of indices

function setReveal(pct){
  pct = Math.max(0, Math.min(100, pct));
  viewer.style.setProperty("--reveal", pct + "%");
  slider.value = pct;
}

function loadIndex(i){
  const list = filteredIndices ?? [...Array(manifest.items.length).keys()];
  const n = list.length;
  if(n === 0) return;

  // i is position in filtered list
  const pos = (i + n) % n;
  const realIdx = list[pos];

  idx = realIdx;
  const it = manifest.items[idx];

  nameLabel.textContent = it.name;
  countLabel.textContent = `${pos+1} / ${n}（总 ${manifest.items.length}）`;

  if(!swapped){
    imgA.src = it.a;
    imgB.src = it.b;
    leftLabel.textContent = "左：Before";
    rightLabel.textContent = "右：Dehazed";
  }else{
    imgA.src = it.b;
    imgB.src = it.a;
    leftLabel.textContent = "左：Dehazed";
    rightLabel.textContent = "右：Before";
  }

  // sync select to real index
  sel.value = String(idx);
  setActiveThumb(idx);
}

function getCurrentPosInFiltered(){
  const list = filteredIndices ?? [...Array(manifest.items.length).keys()];
  const pos = list.indexOf(idx);
  return pos >= 0 ? pos : 0;
}

function next(){
  loadIndex(getCurrentPosInFiltered() + 1);
}
function prev(){
  loadIndex(getCurrentPosInFiltered() - 1);
}

document.getElementById("nextBtn").onclick = next;
document.getElementById("prevBtn").onclick = prev;
document.getElementById("swapBtn").onclick = () => {
  swapped = !swapped;
  // reload current (keep same idx)
  const pos = getCurrentPosInFiltered();
  loadIndex(pos);
};

slider.addEventListener("input", (e)=> setReveal(Number(e.target.value)));

function clientXToPct(clientX){
  const r = viewer.getBoundingClientRect();
  const x = Math.max(r.left, Math.min(r.right, clientX));
  return ((x - r.left) / r.width) * 100;
}

let dragging = false;
viewer.addEventListener("pointerdown", (e)=>{
  dragging = true;
  viewer.setPointerCapture(e.pointerId);
  setReveal(clientXToPct(e.clientX));
});
viewer.addEventListener("pointermove", (e)=>{
  if(!dragging) return;
  setReveal(clientXToPct(e.clientX));
});
viewer.addEventListener("pointerup", ()=> dragging = false);
viewer.addEventListener("pointercancel", ()=> dragging = false);

document.addEventListener("keydown", (e)=>{
  if(!manifest) return;
  if(e.key === "ArrowRight") setReveal(Number(slider.value) + 2);
  else if(e.key === "ArrowLeft") setReveal(Number(slider.value) - 2);
  else if(e.key === "ArrowDown") next();
  else if(e.key === "ArrowUp") prev();
  else if(e.key === "s" || e.key === "S"){
    swapped = !swapped;
    const pos = getCurrentPosInFiltered();
    loadIndex(pos);
  }
});

function renderThumbs(){
  thumbList.innerHTML = "";
  const items = manifest.items;

  sideCount.textContent = filteredIndices ? `${filteredIndices.length} / ${items.length}` : `${items.length}`;

  (filteredIndices ?? [...Array(items.length).keys()]).forEach((realIdx) => {
    const it = items[realIdx];
    const div = document.createElement("div");
    div.className = "thumb-item";
    div.dataset.idx = String(realIdx);

    // thumb 优先；没有就用 a
    const thumbSrc = it.thumb ? it.thumb : it.a;

    div.innerHTML = `
      <img loading="lazy" src="${thumbSrc}" alt="${it.name}"/>
      <div>
        <div class="thumb-name" title="${it.name}">${it.name}</div>
        <div class="thumb-sub">${it.thumb ? "thumb" : "A图预览"}</div>
      </div>
    `;

    div.addEventListener("click", () => {
      // 点击切换到该 realIdx 对应项：先算它在 filtered 里的位置
      const list = filteredIndices ?? [...Array(items.length).keys()];
      const pos = list.indexOf(realIdx);
      loadIndex(pos >= 0 ? pos : 0);
    });

    thumbList.appendChild(div);
  });

  setActiveThumb(idx);
}

function setActiveThumb(realIdx){
  const nodes = thumbList.querySelectorAll(".thumb-item");
  nodes.forEach(n => n.classList.remove("active"));
  const cur = thumbList.querySelector(`.thumb-item[data-idx="${realIdx}"]`);
  if(cur){
    cur.classList.add("active");
    cur.scrollIntoView({block: "nearest"});
  }
}

function applyFilter(q){
  q = (q || "").trim().toLowerCase();
  if(!q){
    filteredIndices = null;
  }else{
    filteredIndices = manifest.items
      .map((it, i) => ({it, i}))
      .filter(x => x.it.name.toLowerCase().includes(q))
      .map(x => x.i);
  }
  renderThumbs();
  // 过滤后切到第一张
  loadIndex(0);
}

thumbSearch.addEventListener("input", (e) => applyFilter(e.target.value));

async function main(){
  const resp = await fetch("manifest.json");
  manifest = await resp.json();

  // select：用真实索引
  sel.innerHTML = "";
  manifest.items.forEach((it, i)=>{
    const opt = document.createElement("option");
    opt.value = String(i);
    opt.textContent = it.name;
    sel.appendChild(opt);
  });

  sel.addEventListener("change", ()=>{
    // 选择后切到该 realIdx 在 filtered 的位置
    const realIdx = Number(sel.value);
    const list = filteredIndices ?? [...Array(manifest.items.length).keys()];
    const pos = list.indexOf(realIdx);
    loadIndex(pos >= 0 ? pos : 0);
  });

  setReveal(50);
  renderThumbs();
  loadIndex(0);
}
main();
</script>
</body>
</html>
"""


def to_uint8_rgb(arr: np.ndarray, pmin: float, pmax: float) -> np.ndarray:
    """
    Convert arbitrary TIFF array to uint8 RGB for web preview.
    - If 2D: normalize and repeat to RGB
    - If 3D: infer channel placement, take first 3 channels, normalize per-channel
    """
    arr = np.asarray(arr)

    # Handle shape (C,H,W) -> (H,W,C)
    if arr.ndim == 3:
        if arr.shape[0] <= 8 and arr.shape[2] > 8 and arr.shape[1] > 8:
            # likely (C,H,W)
            arr = np.transpose(arr, (1, 2, 0))

    if arr.ndim == 2:
        x = arr.astype(np.float32)
        lo, hi = np.percentile(x, [pmin, pmax])
        if hi <= lo:
            lo, hi = float(np.min(x)), float(np.max(x)) + 1e-6
        x = np.clip((x - lo) / (hi - lo), 0, 1)
        x = (x * 255.0 + 0.5).astype(np.uint8)
        rgb = np.repeat(x[:, :, None], 3, axis=2)
        return rgb

    if arr.ndim == 3:
        c = arr.shape[2]
        if c == 1:
            x = arr[:, :, 0]
            return to_uint8_rgb(x, pmin, pmax)
        # take first 3 bands
        x = arr[:, :, :3].astype(np.float32)
        out = np.zeros_like(x, dtype=np.uint8)
        for k in range(3):
            band = x[:, :, k]
            lo, hi = np.percentile(band, [pmin, pmax])
            if hi <= lo:
                lo, hi = float(np.min(band)), float(np.max(band)) + 1e-6
            y = np.clip((band - lo) / (hi - lo), 0, 1)
            out[:, :, k] = (y * 255.0 + 0.5).astype(np.uint8)
        return out

    raise ValueError(f"Unsupported TIFF array shape: {arr.shape}")


def read_tiff(path: Path):
    # Prefer tifffile for geo/remote-sensing tiffs
    if tiff is not None:
        return tiff.imread(str(path))
    # Fallback: PIL (may fail on some TIFFs)
    with Image.open(path) as im:
        return np.array(im)


def save_image_uint8(rgb: np.ndarray, out_path: Path, fmt: str, quality: int):
    img = Image.fromarray(rgb, mode="RGB")
    out_path.parent.mkdir(parents=True, exist_ok=True)

    fmt_lower = fmt.lower()
    if fmt_lower == "png":
        img.save(out_path, format="PNG", optimize=True)
    elif fmt_lower in ("jpg", "jpeg"):
        img.save(out_path, format="JPEG", quality=quality, subsampling=0, optimize=True)
    elif fmt_lower == "webp":
        img.save(out_path, format="WEBP", quality=quality, method=6)
    else:
        raise ValueError("fmt must be png/jpg/webp")

def collect_pairs(dir_a: Path, dir_b: Path, recursive: bool, b_suffix: str):
    exts = {".tif", ".tiff", ".TIF", ".TIFF"}

    def iter_files(d: Path):
        if recursive:
            for p in d.rglob("*"):
                if p.is_file() and p.suffix in exts:
                    yield p
        else:
            for p in d.iterdir():
                if p.is_file() and p.suffix in exts:
                    yield p

    # A: stem -> path
    a_map = {}
    for p in iter_files(dir_a):
        a_map.setdefault(p.stem, p)

    # B: strip suffix, base_stem -> path
    b_map = {}
    for p in iter_files(dir_b):
        stem = p.stem
        if stem.endswith(b_suffix):
            base = stem[: -len(b_suffix)]
            b_map.setdefault(base, p)

    common = sorted(set(a_map.keys()) & set(b_map.keys()))
    pairs = [(stem, a_map[stem], b_map[stem]) for stem in common]
    return pairs

In [ ]:
# 生成png

fmt = "png"  # choices=["webp", "png", "jpg"], help="Output image format for web"
quality = 90  # JPG/WebP quality"
pmin = 0  # Percentile min for normalization
pmax = 100  # Percentile max for normalization
recursive = False  # Whether to search subdirectories for TIFFs
b_suffix = "_dehaze"  # Suffix appended to A's filename stem in folder B

dir_a = Path(r"E:\\project\\cloud_seg_compare_page\\test_patch")
dir_b = Path(r"E:\\project\\cloud_seg_compare_page\\test_patch\\dehazed")
out_dir = Path(r"E:\\project\\cloud_seg_compare_page\\test_patch")
assets_a = out_dir / "assets" / "A"
assets_b = out_dir / "assets" / "B"
thumb_dir = out_dir / "assets" / "thumbs"

pairs = collect_pairs(dir_a, dir_b, recursive, b_suffix)
if not pairs:
    raise SystemExit("No matched TIFF pairs found by filename stem.")

items = []
for i, (stem, pa, pb) in enumerate(pairs, 1):
    print(f"[{i}/{len(pairs)}] {stem}")

    arr_a = read_tiff(pa)
    arr_b = read_tiff(pb)

    rgb_a = to_uint8_rgb(arr_a, pmin, pmax)
    rgb_b = to_uint8_rgb(arr_b, pmin, pmax)

    out_a = assets_a / f"{stem}.{fmt}"
    out_b = assets_b / f"{stem}.{fmt}"

    save_image_uint8(rgb_a, out_a, fmt, quality)
    save_image_uint8(rgb_b, out_b, fmt, quality)

    items.append({
        "name": stem,
        "a": str(Path("assets") / "A" / out_a.name).replace("\\", "/"),
        "b": str(Path("assets") / "B" / out_b.name).replace("\\", "/"),
        "thumb": str(Path("assets") / "thumbs" / out_a.name).replace("\\", "/"),
    })




[1/26] CB04A_WPM_E105.4_N37.3_20250113_L4D0000707690_RGB_105.80_37.50_tile_3_1


C:\Users\mutongyao\AppData\Local\Temp\ipykernel_16776\3773037077.py:334: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(rgb, mode="RGB")


[2/26] CB04A_WPM_E108.5_N34.3_20250611_L4D0000770228_RGB_108.80_34.20_tile_1_2
[3/26] CB04A_WPM_E110.9_N42.4_20250712_L4D0000783106_RGB_110.70_42.10
[4/26] CB04A_WPM_E112.4_N36.5_20250610_L4D0000769623_RGB_112.40_36.40
[5/26] CB04A_WPM_E112.8_N29.9_20250716_L4D0000784832_RGB_113.10_29.30
[6/26] CB04A_WPM_E118.2_N38.0_20250403_L4D0000740393_RGB_118.40_37.60
[7/26] CB04A_WPM_E120.5_N41.0_20250915_L4D0000811708_RGB_120.80_40.70
[8/26] CB04A_WPM_E121.6_N39.5_20250925_L4D0000815557_RGB_121.70_39.00
[9/26] CB04A_WPM_E124.4_N47.6_20250624_L4D0000775273_RGB_124.20_47.20
[10/26] CB04A_WPM_E83.2_N34.3_20241202_L4D0000689516_RGB_82.90_33.90_tile_2_3
[11/26] CB04A_WPM_E83.6_N43.2_20250723_L4D0000788180_RGB_83.20_43.00
[12/26] GF1B_PMS_E112.3_N37.4_20250714_L4D1228737783_RGB_112.50_37.70
[13/26] GF1D_PMS_E126.0_N44.1_20250119_L4D1257545459_RGB_126.30_44.20_tile_2_1
[14/26] GF1D_PMS_E126.0_N44.1_20250119_L4D1257545459_RGB_126.30_44.20_tile_4_1
[15/26] GF1_PMS1_E113.0_N29.1_20250607_L4D13909771001_RG

In [15]:
import json
from pathlib import Path
from PIL import Image

def main(root="viewer", size=240, quality=85, fmt=None):
    root = Path(root)
    manifest_path = root / "manifest.json"
    assert manifest_path.exists(), f"manifest not found: {manifest_path}"

    manifest = json.loads(manifest_path.read_text(encoding="utf-8"))
    items = manifest.get("items", [])
    assert items, "manifest.items is empty"

    thumbs_dir = root / "assets" / "thumbs"
    thumbs_dir.mkdir(parents=True, exist_ok=True)

    for i, it in enumerate(items, 1):
        src_rel = Path(it["a"])  # e.g. assets/A/aaa.webp
        src = root / src_rel
        if not src.exists():
            print(f"[skip] missing source: {src}")
            continue

        # 输出格式：默认跟源图一致；也可强制 fmt="webp"/"png"/"jpg"
        out_ext = (fmt or src.suffix.lstrip(".")).lower()
        if out_ext == "jpeg":
            out_ext = "jpg"

        out_name = f"{it['name']}.{out_ext}"
        out_rel = Path("assets") / "thumbs" / out_name
        out_path = root / out_rel

        with Image.open(src) as im:
            im = im.convert("RGB")
            im.thumbnail((size, size))

            if out_ext == "png":
                im.save(out_path, format="PNG", optimize=True)
            elif out_ext == "jpg":
                im.save(out_path, format="JPEG", quality=quality, subsampling=0, optimize=True)
            elif out_ext == "webp":
                im.save(out_path, format="WEBP", quality=quality, method=6)
            else:
                raise ValueError(f"Unsupported thumb format: {out_ext}")

        it["thumb"] = str(out_rel).replace("\\", "/")
        if i % 50 == 0:
            print(f"[{i}/{len(items)}] done")

    manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
    print("OK. thumbs generated at:", thumbs_dir)
    print("manifest updated:", manifest_path)


main(root="E:\\project\\cloud_seg_compare_page\\test_patch", size=240, quality=85, fmt=None)


OK. thumbs generated at: E:\project\cloud_seg_compare_page\test_patch\assets\thumbs
manifest updated: E:\project\cloud_seg_compare_page\test_patch\manifest.json


In [23]:
out_dir.mkdir(parents=True, exist_ok=True)
(out_dir / "index.html").write_text(HTML_TEMPLATE, encoding="utf-8")
(out_dir / "manifest.json").write_text(json.dumps({"items": items}, ensure_ascii=False, indent=2), encoding="utf-8")

print("\nDone.")




Done.
